In [0]:
%sql
create catalog pratice_catalog1
managed location 'abfss://data@storagescdsa.dfs.core.windows.net/catalog'

In [0]:
from pyspark.sql.functions import *
customer_schema='customer_id int,first_name string,last_name string,email string,phone string,address string'
customer_df=spark.read.format('csv').schema(customer_schema).option('header',True).load('abfss://data@storagescdsa.dfs.core.windows.net/catalog/landing/customers').withColumn("valid_to", lit(None)) \
                  .withColumn("is_current", lit(1))
customer_updates_df = (
    spark.read
    .format("csv")
    .option("header", "true")
    .schema(customer_schema)
    .load("abfss://data@storagescdsa.dfs.core.windows.net/catalog/landing/customer_updates/*")
)
display(customer_updates_df)

customer_id,first_name,last_name,email,phone,address
101,John,Doe,john.new@example.com,9876543211,Delhi
105,Priya,Singh,priya.singh@xyz.com,9812345678,Mumbai
121,Aarav,Kapoor,aarav.kapoor@example.com,9123456780,Pune
110,null,Sharma,sharmaatexample.com,12345,Chennai
101,Johnny,dew,john.dew@example.com,9876543211,Delhi
101,Johnny,dew,john.new@example.com,9876543211,Delhi


In [0]:
from pyspark.sql import *
combined_df=customer_df.alias('target').join(customer_updates_df.alias('source'),customer_df.customer_id==customer_updates_df.customer_id,how='outer')
changes_df=combined_df.filter((col('target.customer_id')==col('source.customer_id'))&((col('source.first_name').isNotNull())&(col('source.last_name').isNotNull())&(col('source.email').isNotNull())&(col('source.phone').isNotNull())&(col('source.address').isNotNull())))
non_changes_df=combined_df.filter(col('source.customer_id').isNull()).select('target.*')
new_changes_df=changes_df.select('source.*')
window_size=Window.partitionBy('customer_id').orderBy(col('_metadata.file_modification_time').desc())
new_changes_df=new_changes_df.withColumn('rank',dense_rank().over(window_size))
new_changes_df=new_changes_df.withColumn('valid_to',when(col('rank')>1, current_date()).otherwise(lit(None)))
new_changes_df=new_changes_df.withColumn('is_current',when(col('rank')>1, lit(0)).otherwise(lit(1))).drop('rank')
old_changes_df=changes_df.select('target.*').distinct().withColumn('valid_to',current_date()).withColumn('is_current',lit(0))
customer_master_df=non_changes_df.unionByName(new_changes_df).unionByName(old_changes_df)
display(customer_master_df.orderBy('customer_id'))

customer_id,first_name,last_name,email,phone,address,valid_to,is_current
101,Johnny,dew,john.dew@example.com,9876543211,Delhi,null,1
101,John,Doe,user1@mail.com,900000001,Pune,2025-11-06,0
101,Johnny,dew,john.new@example.com,9876543211,Delhi,2025-11-06,0
101,John,Doe,john.new@example.com,9876543211,Delhi,2025-11-06,0
102,Amit,Sharma,user2@mail.com,900000002,Pune,null,1
103,Priya,Singh,user3@mail.com,900000003,Mumbai,null,1
104,Sara,Khan,user4@mail.com,900000004,Delhi,null,1
105,Mike,Smith,user5@mail.com,900000005,Delhi,2025-11-06,0
105,Priya,Singh,priya.singh@xyz.com,9812345678,Mumbai,null,1
106,Nina,Patel,user6@mail.com,900000006,Pune,null,1


In [0]:
%sql
MERGE INTO customer1 AS target
USING customer_update_v1 AS source
ON target.customer_id = source.customer_id AND target.type1 = 1

WHEN MATCHED AND (
    target.first_name != source.first_name OR
    target.last_name  != source.last_name OR
    target.email      != source.email OR
    target.phone      != source.phone OR
    target.address    != source.address
)
THEN UPDATE SET
    target.valid_to = current_date,
    target.type1 = 0

WHEN NOT MATCHED THEN
INSERT (
    customer_id,
    first_name,
    last_name,
    email,
    phone,
    address,
    valid_to,
    type1
)
VALUES (
    source.customer_id,
    source.first_name,
    source.last_name,
    source.email,
    source.phone,
    source.address,
    NULL,
    1
)


num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
4,3,0,1


In [0]:
%sql
select * from customer1 order by customer_id;

customer_id,first_name,last_name,email,phone,address,valid_to,type1
101,John,Doe,user1@mail.com,900000001,Pune,2025-11-05,0
102,Amit,Sharma,user2@mail.com,900000002,Pune,null,1
103,Priya,Singh,user3@mail.com,900000003,Mumbai,null,1
104,Sara,Khan,user4@mail.com,900000004,Delhi,null,1
105,Mike,Smith,user5@mail.com,900000005,Delhi,2025-11-05,0
106,Nina,Patel,user6@mail.com,900000006,Pune,null,1
107,Ravi,Verma,user7@mail.com,900000007,Delhi,null,1
108,Simran,Kaur,user8@mail.com,900000008,Mumbai,null,1
109,Yash,Gupta,user9@mail.com,900000009,Bangalore,null,1
110,Emma,Jones,user10@mail.com,900000010,Delhi,2025-11-05,0
